# Training

In [1]:
import sys, os
import pandas as pd
import numpy as np
import torch
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
sys.path.append("../../../../../")
from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf, trans_DF_to_NP_by_windowNum
from Clust.clust.ML.common import ML_pipeline, tool
from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient



db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


## 1. Data Preparation

### 1-1. Parameter Setting

In [2]:
# 현재 Classification의 경우 window_split만 검증 완료 하였음

app_name='actionPattern'
model_purpose = 'classification'

feature_X_list = ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8']
feature_y_list = ['value']

split_mode = "windows_split"
data_y_flag = True

step = 'train'
bucket_name = 'integration' 
data_clean_level = 0

In [3]:
all_integrated_ms_list = db_client.measurement_list(bucket_name)
print(all_integrated_ms_list)
print("==========================================================")
collection_list = mongo_client.get_collection_list(bucket_name)
print(collection_list)

['classification_actionPattern_cleanLevel0_testX', 'classification_actionPattern_cleanLevel0_testy', 'classification_actionPattern_cleanLevel0_trainX', 'classification_actionPattern_cleanLevel0_trainy', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel0_testX', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel0_trainX', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX', 'forecasting_strawberryOpen_cleanLevel0_testX', 'forecasting_strawberryOpen_cleanLevel0_trainX', 'forecasting_strawberryOpen_cleanLevel4_testX', 'forecasting_strawberryOpen_cleanLevel4_trainX', 'regression_energy_cleanLevel0_testX', 'regression_energy_cleanLevel0_testy', 'regression_energy_cleanLevel0_trainX', 'regression_energy_cleanLevel0_trainy', 'regression_energy_cleanLevel4_testX', 'regression_energy_cleanLevel4_testy', 'regression_energy_cleanLevel4_trainX', 'regression_energy_cleanLevel4_trainy']
['forecasting_Hs2SwineFarmWithWeatherTime

### 1-2. Data Ingestion
#### 1-2-1. Select data name 

In [4]:
dataset_name = model_purpose + '_' + app_name  
data_name_X = dataset_name + '_cleanLevel' + str(data_clean_level)+'_'+step+'X'
data_name_y = dataset_name+'_cleanLevel' + str(data_clean_level)+'_'+ step+'y'
data_meta = mongo_client.get_document_by_json('integration', dataset_name, {'ms_name':data_name_X})[0]

In [5]:
data_name_X, data_name_y

('classification_actionPattern_cleanLevel0_trainX',
 'classification_actionPattern_cleanLevel0_trainy')

In [6]:
data_meta

{'bucket_name': 'integration',
 'collection_name': 'classification_actionPattern',
 'ms_name': 'classification_actionPattern_cleanLevel0_trainX',
 'ingestion_type': 'multiple_ms_by_time',
 'ingestion_param': {'ms_list_info': [['bio_action_sensors',
    'accelerationTrain']],
  'start_time': '1992-01-11',
  'end_time': '2012-02-27'},
 'processing_type': 'step_3',
 'process_param': {'refine_param': {'removeDuplication': {'flag': False},
   'staticFrequency': {'flag': False, 'frequency': None}},
  'outlier_param': {'certainErrorToNaN': {'flag': False},
   'unCertainErrorToNaN': {'flag': False}},
  'imputation_param': {'flag': False}},
 'integration_param': {'integration_frequency': 675,
  'param': {},
  'method': 'meta',
  'integration_duration': 'common'},
 'clean_level': 0}

#### 1-2-2. X-y Data Ingestion pipeline

In [7]:
# Data Ingestion
ingestion_method = 'ms_all'
ingestion_param_X = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_X,
    'feature_list' : feature_X_list                              
}
ingestion_param_y = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_y,
    'feature_list' : feature_y_list                              
}

In [8]:
data_X, data_y = ML_pipeline.Xy_data_preparation(ingestion_param_X, data_y_flag, ingestion_param_y, ingestion_method, db_client)

In [9]:
print(len(data_X))
print(len(data_y))
print(data_X.shape)
print(data_y.shape)

941056
7352
(941056, 9)
(7352, 1)


In [10]:
data_X

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8
time,,,,,,,,,
1992-01-11 00:00:00+00:00,0.000181,0.010767,0.055561,0.030191,0.066014,0.022859,1.012817,-0.123217,0.102934
1992-01-11 00:11:15+00:00,0.010139,0.006579,0.055125,0.043711,0.042699,0.010316,1.022833,-0.126876,0.105687
1992-01-11 00:22:30+00:00,0.009276,0.008929,0.048405,0.035688,0.074850,0.013250,1.022028,-0.124004,0.102102
1992-01-11 00:33:45+00:00,0.005066,0.007489,0.049775,0.040402,0.057320,0.017751,1.017877,-0.124928,0.106553
1992-01-11 00:45:00+00:00,0.010810,0.006141,0.043013,0.047097,0.052343,0.002553,1.023680,-0.125767,0.102814
...,...,...,...,...,...,...,...,...,...
2012-02-26 23:03:45+00:00,0.022358,-0.280075,-0.180578,0.973228,1.083094,-0.226884,0.991497,-0.486260,-0.205803
2012-02-26 23:15:00+00:00,-0.024451,-0.248612,-0.153920,1.004266,1.187832,-0.313591,0.945067,-0.453405,-0.180733
2012-02-26 23:26:15+00:00,-0.071907,-0.194322,-0.127555,1.004855,1.156645,-0.362512,0.898095,-0.397775,-0.156105


In [11]:
data_y

,value
time,
1992-01-11 00:00:00+00:00,4.0
1992-01-12 00:00:00+00:00,4.0
1992-01-13 00:00:00+00:00,4.0
1992-01-14 00:00:00+00:00,4.0
1992-01-15 00:00:00+00:00,4.0
...,...
2012-02-22 00:00:00+00:00,1.0
2012-02-23 00:00:00+00:00,1.0
2012-02-24 00:00:00+00:00,1.0


### 1-2-2. Random Nan Insert (Test)

In [12]:
nan_ratio = 0.00

In [13]:
data_X = tool.random_nan_df(data_X, nan_ratio)
data_y = tool.random_nan_df(data_y, nan_ratio)

../../../../Clust/clust/ML/common/tool.py:17: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.loc[df.sample(frac=nan_ratio). index, col] = pd.np.nan


#### 1-2-3. Data scaling

In [14]:
scaler_param='scale'
scale_method='minmax'
scaler_path = './scaler/'

In [15]:
dataX_scaled, X_scalerFilePath, datay_scaled, y_scalerFilePath= ML_pipeline.Xy_data_scaling_train(data_name_X, data_X, data_name_y, data_y, scaler_path, scaler_param, scale_method)

['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8']
Make New scaler File
['value']
Make New scaler File


## 2. Cleaning and split
### 2.1 pipeline - clean low quality column

In [16]:
model_clean = False
nan_process_info = {'type':'num', 'ConsecutiveNanLimit':10000, 'totalNaNLimit':100000}

In [17]:
dataX_scaled = ML_pipeline.clean_low_quality_column(model_clean, nan_process_info, dataX_scaled)
feature_X_list= list(dataX_scaled.columns)

### 2.2 Train/Val Split pipeline

In [18]:
split_ratio = 0.8

In [19]:
# TODO 데이터 나뉘는 부분 추가로 작성된 것 지수님에게 물어봐야 함
day_window_size = tool.get_default_day_window_size(dataX_scaled)
train_x, val_x, train_y, val_y = ML_pipeline.split_data_by_mode(split_mode, split_ratio, dataX_scaled, datay_scaled, day_window_size)

In [20]:
day_window_size

128

In [21]:
train_x

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8
time,,,,,,,,,
1992-01-11 00:00:00+00:00,0.486708,0.584184,0.584082,0.535918,0.515370,0.543161,0.555284,0.521129,0.596563
1992-01-11 00:11:15+00:00,0.490641,0.582380,0.583903,0.537439,0.513381,0.540716,0.559043,0.519822,0.597505
1992-01-11 00:22:30+00:00,0.490300,0.583392,0.581139,0.536537,0.516124,0.541288,0.558741,0.520847,0.596278
1992-01-11 00:33:45+00:00,0.488638,0.582772,0.581703,0.537067,0.514629,0.542166,0.557183,0.520517,0.597802
1992-01-11 00:45:00+00:00,0.490906,0.582191,0.578922,0.537820,0.514204,0.539202,0.559361,0.520218,0.596521
...,...,...,...,...,...,...,...,...,...
2008-02-16 23:03:45+00:00,0.487328,0.582224,0.561860,0.531996,0.508964,0.542586,0.557988,0.504818,0.568865
2008-02-16 23:15:00+00:00,0.486507,0.581658,0.561933,0.532397,0.508276,0.541986,0.557211,0.504345,0.568898
2008-02-16 23:26:15+00:00,0.487463,0.581152,0.563547,0.533299,0.508082,0.541137,0.558124,0.503923,0.570212


### 2.3 Data Transformation & Clean2 pipeline

In [22]:
max_nan_limit_ratio = 0.5
transformParameter = {
        'past_step':day_window_size,
        'max_nan_limit_ratio': max_nan_limit_ratio
}

In [23]:
# train_X_array, train_y_array = ML_pipeline.transform_data_by_split_mode(split_mode, transformParameter, train_x, train_y)
# val_X_array, val_y_array = ML_pipeline.transform_data_by_split_mode(split_mode, transformParameter, val_x, val_y)

In [24]:
model_method = 'LSTM_cf'
dim = 3

if model_method == "FC_cf":
    dim = 2

if type(train_x) !=  np.ndarray:
    train_X_array, train_y_array = trans_DF_to_NP_by_windowNum(train_x, train_y, transformParameter)
    val_X_array, val_y_array = trans_DF_to_NP_by_windowNum(val_x, val_y, transformParameter)
    
input_size = train_X_array.shape[2]
# TBD
#if dim != 2:
seq_len = train_X_array.shape[1] # seq_length

window_size: 128 nan_limit_num: 64
(752768, 9) (5881, 128, 9)
(5881, 1) (5881, 1)
window_size: 128 nan_limit_num: 64
(188288, 9) (1471, 128, 9)
(1471, 1) (1471, 1)


In [25]:
print(train_X_array.shape)
print(train_y_array.shape)
print(val_X_array.shape)
print(val_y_array.shape)

(5881, 128, 9)
(5881, 1)
(1471, 128, 9)
(1471, 1)


In [26]:
# model_method = 'LSTM_cf'
# dim = 3

# if model_method == "FC_cf":
#     dim = 2

# input_size = train_X_array.shape[1]
# if dim != 2:
#     seq_len = train_X_array.shape[2] # seq_length

In [27]:
type(train_X_array)

numpy.ndarray

### 2.4 Set Model and train parameter

In [28]:
# RNN models (RNN, LSTM, GRU) parameters
if model_method == 'LSTM_cf' or model_method == 'GRU_cf':
    modelParameter = {
        'input_size': input_size,
        'seq_len': seq_len,
        'num_classes': 6,
        'num_layers': 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
        'hidden_size': 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
        'dropout': 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
        'bidirectional': True  # 모델의 양방향성 여부, bool(default: True)   
    }
    if model_method == 'LSTM_cf':
        modelParameter['rnn_type'] = 'lstm'
    else:
        modelParameter['rnn_type'] = 'gru'
        
# CNN_1D model parameters
elif model_method == 'CNN_1D_cf':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'num_classes': 6,
    'output_channels': 64, # convolution layer의 output channel, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
    'kernel_size': 3, # convolutional layer의 filter 크기, int(default: 3, 범위: 3 이상, 홀수로 설정 권장)
    'stride': 1, # convolution layer의 stride 크기, int(default: 1, 범위: 1 이상)
    'padding': 0, # padding 크기, int(default: 0, 범위: 0 이상)
    'drop_out': 0.1 # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
    }
    
# LSTM_FCNs model parameters
elif model_method == 'LSTM_FCNs_cf':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'num_classes': 6,
    'num_layers': 1,  # recurrent layers의 수, int(default: 1, 범위: 1 이상)
    'lstm_drop_out': 0.4, # LSTM dropout 확률, float(default: 0.4, 범위: 0 이상 1 이하)
    'fc_drop_out': 0.1 # FC dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
    }
    
# FC model parameters
elif model_method == 'FC_cf':
    input_size = seq_len * input_size
    modelParameter = {
    'input_size': input_size,
    'num_classes': 6,
    'drop_out': 0.1, # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
    'bias': True# bias 사용 여부, bool(default: True)
    }

train_parameter = {
    'lr': 1e-4,
    'weight_decay': 1e-6, 
    'device': device, 
    'n_epochs': 5, 
    'batch_size': 16
}

In [29]:
model_name = None
model_file_path = None

### 2.5 Set Model name and path pipeline

In [30]:
# model_name and file path check and create
model_name = tool.get_default_model_name(model_name, app_name, model_method, model_clean)
model_file_path = tool.get_default_model_path(model_name, data_name_X, model_method, train_parameter)

./Models/LSTM_cf/actionPattern_LSTM_cf_False/classification_actionPattern_cleanLevel0_trainX/035b06e2d9df4a58aa1dd8622746dd1f/model.pkl


In [31]:
model_name

'actionPattern_LSTM_cf_False'

In [32]:
from Clust.clust.ML.classification.train import ClassificationTrain as CML
def CLUST_classification_train(train_X_array, train_y_array, val_X_array, val_y_array, train_parameter, model_method, model_file_path, modelParameter):
    cml = CML()
    cml.set_param(train_parameter)
    cml.set_model(model_method, modelParameter)
    cml.set_data(train_X_array, train_y_array, val_X_array, val_y_array)
    cml.train()
    cml.save_best_model(model_file_path)


In [33]:
CLUST_classification_train(train_X_array, train_y_array, val_X_array, val_y_array, train_parameter, model_method, model_file_path, modelParameter)

Start training model

Epoch 1/5
train Loss: 0.8013 Acc: 0.7718
val Loss: 0.5130 Acc: 0.8001

Training complete in 0m 30s
Best val Acc: 0.975962


## 4. save metadata

In [34]:
# from Clust.clust.transformation.general.dataScaler import encode_hash_style
modelTags =["model_tag_example"]
trainDataType = "timeseries"

modelInfoMeta ={
    "trainDataInfo":data_meta,
    "modelName":model_name,
    "dataSplitMode":split_mode,
    "featureXList":feature_X_list,
    "featureyList":feature_y_list,
    "data_y_flag": data_y_flag,
    "trainDataType":trainDataType,
    "modelPurpose":model_purpose,
    "modelMethod":model_method,
    "modelTags":modelTags,
    "modelCleanLevel":model_clean,
    "trainParameter": train_parameter,
    "modelParameter": modelParameter,
    "transformParameter":transformParameter,
    "scalerParam":scaler_param,
    "trainDataName":[data_name_X, data_name_y], 

    "files":{
        "modelFile":{
            "fileName":"model.pth",
            "filePath":model_file_path
        },
        "XScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":X_scalerFilePath       
        },
        "yScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":y_scalerFilePath      
        }
    }
}

In [35]:
from Clust.clust.ML.tool import meta as ml_meta
modelInfoMeta = ml_meta.save_model_meta_data(mongo_client, modelInfoMeta)

========== Data Save Success ==========
{'trainDataInfo': {'bucket_name': 'integration', 'collection_name': 'classification_actionPattern', 'ms_name': 'classification_actionPattern_cleanLevel0_trainX', 'ingestion_type': 'multiple_ms_by_time', 'ingestion_param': {'ms_list_info': [['bio_action_sensors', 'accelerationTrain']], 'start_time': '1992-01-11', 'end_time': '2012-02-27'}, 'processing_type': 'step_3', 'process_param': {'refine_param': {'removeDuplication': {'flag': False}, 'staticFrequency': {'flag': False, 'frequency': None}}, 'outlier_param': {'certainErrorToNaN': {'flag': False}, 'unCertainErrorToNaN': {'flag': False}}, 'imputation_param': {'flag': False}}, 'integration_param': {'integration_frequency': 675, 'param': {}, 'method': 'meta', 'integration_duration': 'common'}, 'clean_level': 0}, 'modelName': 'actionPattern_LSTM_cf_False', 'dataSplitMode': 'windows_split', 'featureXList': ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8'], 'featureyLis

In [36]:
modelInfoMeta